In [1]:
%reload_ext autoreload
%autoreload 2

# Mahakala

In [2]:
import mahakala as ma

import numpy as np
import matplotlib.pyplot as plt

cpu


## Geodesic trajectories
example demo showing the geodesic trajectories for a non-spinning black hole

In [3]:
s0 = ma.initialize_geodesics_at_camera(60,1000,-20,20,50,'Grid',0.9)

In [4]:
S,final_dt = ma.geodesic_integrator(10000,s0,40,1e-4, .9)


 11%|█         | 1115/10000 [00:16<02:08, 68.92it/s]


In [ ]:
geodesic_slices_Schwarschild()

## Shadows
example demo of generating a few Kerr shadows for different inclinations and spins

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1)

inc = 30
for a in [0,.3,.5,.7,.9]:
    radii, phi = kerr_shadow(a, inc)
    x = radii*np.cos(phi)
    y = radii*np.sin(phi)
    ax1.plot(x,y, label = r'$a=$'+str(a))
ax1.legend(frameon=False)


for inc in [0,30,60,90]:
    radii, phi = kerr_shadow(a, inc)
    x = radii*np.cos(phi)
    y = radii*np.sin(phi)
    ax2.plot(x,y, label = r'$i=$'+str(inc))
ax2.legend(frameon=False)



## Analytic Accretion Model
example demo of generating an image of an accretion flow based on an analytic accretion model

In [ ]:
# Take in initial parameters, camera position, and parameters

grid_photons = make_grid_photons(inclination_angle, spin, distance)

my_geodesics = geodesics(grid_photons, inclination_angle, spin, distance)

fluid = get_fluid_grid(GRMHD or analytic?)

image = synemiss(my_geodesics, R_high, R_low, Mscale, fluid, ...)

## Simulation Image
example demo of generating an image based on an example GRMHD snapshot